# V2log YOLO26-N Weight Plate Training

**YOLO26-N** = 2026년 1월 출시 최신 모델 (YOLO11 대비 CPU 추론 43% 빠름)

**셀을 위에서 아래로 순서대로 ▶ 클릭하면 됩니다!**

| 셀 | 내용 | 예상 시간 |
|-----|------|----------|
| 1 | Google Drive 연결 | 30초 |
| 2 | 데이터셋 압축 해제 | 1분 |
| 3 | YOLO26 설치 + 설정 | 1분 |
| 4 | **YOLO26-N 학습 시작** | **30분~2시간** |
| 5 | 모델 변환 + 저장 | 2분 |

> ⚠️ **학습 중 브라우저 탭 닫지 마세요!**
> ⚠️ **YOLO11 아닙니다! 반드시 YOLO26-N으로 학습하세요!**

## 셀 1: Google Drive 연결
▶ 실행 → 팝업에서 Google 계정 선택 → 허용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive 연결 완료!')

## 셀 2: 데이터셋 압축 해제
Drive에 업로드한 ZIP을 Colab 로컬에 풀기

In [ ]:
import zipfile
import os
import glob

# Google Drive에서 ZIP 파일 자동 찾기
drive_path = '/content/drive/MyDrive/V2log-CV-Training/'

if not os.path.exists(drive_path):
    print('V2log-CV-Training 폴더가 없습니다!')
    print('Google Drive에 V2log-CV-Training 폴더를 만들고 ZIP을 업로드하세요.')
else:
    # ZIP 파일 자동 감지
    zip_files = glob.glob(os.path.join(drive_path, '*.zip'))
    
    if not zip_files:
        print(f'{drive_path} 에 ZIP 파일이 없습니다!')
        print('폴더 안의 파일들:')
        for f in os.listdir(drive_path):
            print(f'  {f}')
    else:
        # ZIP이 여러 개면 가장 최신 파일 사용
        zip_path = max(zip_files, key=os.path.getmtime)
        print(f'ZIP 발견: {os.path.basename(zip_path)}')
        print('압축 해제 중...')
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('/content/dataset')
        
        # 폴더 구조 확인
        print('\n폴더 구조:')
        for root, dirs, files in os.walk('/content/dataset'):
            level = root.replace('/content/dataset', '').count(os.sep)
            indent = '  ' * level
            print(f'{indent}{os.path.basename(root)}/')
            if level < 2:
                for file in files[:3]:
                    print(f'{indent}  {file}')
                if len(files) > 3:
                    print(f'{indent}  ... 외 {len(files)-3}개')
        
        # 이미지 개수 확인
        total = 0
        for split in ['train', 'valid', 'test']:
            img_dir = f'/content/dataset/{split}/images'
            if os.path.exists(img_dir):
                count = len(os.listdir(img_dir))
                total += count
                print(f'\n{split}: {count}장')
        
        print(f'\n총 {total}장 — 데이터셋 준비 완료!')

## 셀 3: YOLO26 설치 + data.yaml 경로 수정

ultralytics 최신 버전이어야 YOLO26-N을 사용할 수 있습니다.

In [ ]:
# Ultralytics 최신 버전 설치 (YOLO26 지원 필수!)
!pip install ultralytics --upgrade -q

# 버전 확인
import ultralytics
print(f'ultralytics 버전: {ultralytics.__version__}')

# YOLO26 지원 확인 (v8.4.14 이상 필요)
version_parts = ultralytics.__version__.split('.')
major, minor = int(version_parts[0]), int(version_parts[1])
patch = int(version_parts[2]) if len(version_parts) > 2 else 0
if major >= 8 and minor >= 4 and patch >= 14:
    print('YOLO26-N 지원 확인!')
elif major >= 8 and minor >= 5:
    print('YOLO26-N 지원 확인!')
else:
    print(f'경고: v8.4.14 이상 필요! 현재 {ultralytics.__version__}')
    print('!pip install ultralytics --upgrade 를 다시 실행하세요')

import yaml

# data.yaml 경로 수정 (Colab 환경에 맞게)
yaml_path = '/content/dataset/data.yaml'

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# 경로를 Colab 절대 경로로 수정
data['path'] = '/content/dataset'
data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print(f'\n클래스 수: {data["nc"]}')
print(f'클래스: {data["names"]}')
print('\ndata.yaml 경로 수정 완료!')

## 셀 4: YOLO26-N 학습 시작

⏱️ **30분~2시간 걸립니다. 브라우저 탭 닫지 마세요!**

> **반드시 `yolo26n.pt`** 를 사용합니다. yolo11n.pt 절대 사용 금지!
>
> 화면에 epoch 1/100, 2/100 ... 이렇게 진행 상황이 나옵니다.

In [ ]:
from ultralytics import YOLO

# ============================================
#   YOLO26-N 모델 (절대 yolo11n 사용 금지!)
# ============================================
MODEL_NAME = 'yolo26n.pt'

model = YOLO(MODEL_NAME)
print(f'{MODEL_NAME} 모델 로드 완료!')
print(f'모델 정보: {model.info()}')

# 학습 시작
results = model.train(
    data='/content/dataset/data.yaml',
    epochs=100,        # 최대 100번 반복
    imgsz=640,         # 이미지 크기
    batch=16,          # 한 번에 16장씩
    device=0,          # GPU 사용
    patience=20,       # 20번 동안 개선 없으면 조기 종료
    save=True,         # 체크포인트 저장
    project='/content/models',
    name='weight_plate_yolo26n',
    exist_ok=True,
)

# 결과 출력
print(f'\n{"="*50}')
print(f'YOLO26-N 학습 완료!')
print(f'{"="*50}')

mAP50 = results.results_dict.get('metrics/mAP50(B)', 0)
mAP50_95 = results.results_dict.get('metrics/mAP50-95(B)', 0)
print(f'mAP50:    {mAP50:.4f}')
print(f'mAP50-95: {mAP50_95:.4f}')

if mAP50 >= 0.8:
    print('\n80% 이상! 다음 셀(변환)을 실행하세요!')
elif mAP50 >= 0.6:
    print('\n60~80% — 나쁘지 않음. 일단 변환해서 테스트해보세요.')
else:
    print('\n60% 미만 — 데이터 추가가 필요할 수 있습니다.')

## 셀 5: TFLite 변환 + Google Drive에 저장

학습 완료 후 실행! 결과물이 Google Drive에 자동 저장됩니다.

In [ ]:
import shutil
import os
from ultralytics import YOLO

# 학습된 모델 로드
best_model_path = '/content/models/weight_plate_yolo26n/weights/best.pt'
model = YOLO(best_model_path)

# TFLite로 변환 (모바일용)
print('TFLite 변환 중...')
model.export(format='tflite', half=True, imgsz=640)
print('변환 완료!')

# Google Drive에 결과물 복사
dst = '/content/drive/MyDrive/V2log-CV-Training/results_yolo26n/'
os.makedirs(dst, exist_ok=True)

# 모델 파일 복사
weights_dir = '/content/models/weight_plate_yolo26n/weights/'
for f in os.listdir(weights_dir):
    src_file = os.path.join(weights_dir, f)
    if os.path.isfile(src_file):
        shutil.copy2(src_file, dst)
        size_mb = os.path.getsize(src_file) / (1024*1024)
        print(f'  저장: {f} ({size_mb:.1f} MB)')

# 학습 결과 그래프/CSV 복사
results_dir = '/content/models/weight_plate_yolo26n/'
for f in os.listdir(results_dir):
    src_file = os.path.join(results_dir, f)
    if os.path.isfile(src_file) and (f.endswith('.png') or f.endswith('.csv')):
        shutil.copy2(src_file, dst)
        print(f'  저장: {f}')

print(f'\n모든 결과물이 Google Drive에 저장되었습니다!')
print(f'위치: Google Drive > V2log-CV-Training > results_yolo26n')
print(f'\n최종 결과물: best_float16.tflite')
print(f'이 파일을 V2log 앱에 넣으면 무게 자동 감지 완성!')

---
## 완료!

Google Drive > `V2log-CV-Training` > `results_yolo26n` 폴더에 결과물이 저장되었습니다.

| 파일 | 용도 |
|------|------|
| `best.pt` | 학습된 YOLO26-N 모델 (PC용) |
| `best_float16.tflite` | **앱에 넣을 최종 모델** |
| `results.png` | 학습 그래프 |
| `confusion_matrix.png` | 클래스별 정확도 |

**다음 단계**: `best_float16.tflite`를 V2log 앱 `assets/models/`에 넣고 Phase 2B 시작!